In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tokenization_kobert import KoBertTokenizer
import pandas as pd

In [8]:

model = BertForSequenceClassification.from_pretrained("fine_tuned_model", num_labels=3)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

device = torch.device("cpu")
model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [39]:
data = pd.read_csv('all_data.csv')
data

,comment,label
0,좌배 까는건 ㅇㅂ,0
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0
4,재앙이한건햇노,2
...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1
16434,링크도 안박고 가서 글 쓰자고 선동하네,0


In [48]:
data['pred_0'] = None
data['pred_1'] = None
data['pred_2'] = None

data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,None,None,None
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,None,None,None
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,None,None,None
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,None,None,None
4,재앙이한건햇노,2,None,None,None
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,None,None,None
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,None,None,None
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,None,None,None
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,None,None,None


In [49]:
for idx, row in data.iterrows():
    sentence = row['comment']
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #predicted_labels = torch.argmax(logits, dim=1)
    data.iat[idx,2] = logits[0][0].item()
    data.iat[idx,3] = logits[0][1].item()
    data.iat[idx,4] = logits[0][2].item()
    

In [50]:
data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,4.396102,-2.002848,-2.493134
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,4.234729,-1.999449,-2.375609
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,4.081088,-2.403747,-1.749318
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,0.052817,2.119905,-2.698664
4,재앙이한건햇노,2,-1.829934,-2.204185,4.236357
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,-0.782139,3.064341,-2.565609
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,-2.525733,-1.327989,4.015573
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,0.705698,1.268277,-2.471448
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,4.151145,-1.763958,-2.571591


In [51]:
data.to_csv("all_data_pred.csv",encoding = 'cp949', index = False)

In [80]:
import importlib
om = importlib.reload(om)
#import OpenMax as om

In [76]:
ori_label = data['label']
pred_label = list(zip(data['pred_0'],data['pred_1'],data['pred_2']))
pred_label = [[a,b,c] for a,b,c in pred_label]
print(ori_label[:10])
print(pred_label[:10])

0    0
1    0
2    0
3    0
4    2
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
[[4.396101951599121, -2.0028483867645264, -2.493133783340454], [4.234728813171387, -1.9994492530822754, -2.3756091594696045], [4.081088066101074, -2.4037468433380127, -1.7493184804916382], [0.05281704664230347, 2.1199052333831787, -2.6986639499664307], [-1.829933524131775, -2.2041847705841064, 4.23635721206665], [4.289192199707031, -2.208202838897705, -2.1704277992248535], [3.466866970062256, -1.172554850578308, -2.6108405590057373], [1.4613574743270874, 0.8115693926811218, -2.721031427383423], [3.702420234680176, -1.3678532838821411, -2.6241166591644287], [4.273830413818359, -2.1680829524993896, -2.20841646194458]]


In [81]:
openmax = om.OpenMax()
openmax.fit(pred_label,ori_label)

In [110]:
def predict_sentence(sentence):
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = [0,0,0]
    logits[0] = outputs.logits[0][0].item()
    logits[1] = outputs.logits[0][1].item()
    logits[2] = outputs.logits[0][2].item()
    
    om_result = openmax.calculate(logits)
    _class = ['혐오','성차별','일베']
    print("입력문장 : ",sentence)
    print()
    print("모델 예측 결과 - OpenMax 미적용")
    print("혐오 : ",logits[0])
    print("성차별 : ",logits[1])
    print("일베 : ",logits[2])
    print()
    print("모델 예측 결과 - OpenMax 적용")
    print("Unkown : ",om_result[0])
    print("혐오 : ",om_result[1])
    print("성차별 : ",om_result[2])
    print("일베 : ",om_result[3])
    
   

In [121]:
predict_sentence("")

입력문장 :  집가서 밥먹고 싶다

모델 예측 결과 - OpenMax 미적용
혐오 :  3.9857654571533203
성차별 :  -0.7485332489013672
일베 :  -3.6180384159088135

모델 예측 결과 - OpenMax 적용
Unkown :  0.00022974461742028882
혐오 :  0.9637684303563324
성차별 :  0.0180010155043189
일베 :  0.018000809521928462


In [120]:
data['label'].value_counts()

0    10431
1     4481
2     1524
Name: label, dtype: int64

In [133]:
openmax.calculate([3,3,3])

[0.9996296758877599,
 0.00012347946443061738,
 0.00012342245706121604,
 0.0001234221907483572]